In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.feature_selection import SelectFromModel, SelectKBest, SelectPercentile, RFE
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import Ridge, Lasso
import matplotlib.pyplot as plt

In [ ]:
properti_price = pd.read_csv('/Users/saktiyoga/Downloads/Projek UTS_B_matplotlib/Dataset UTS_Gasal 2425.csv')
properti_price.head(100)

In [ ]:
properti_price2 = properti_price.drop('category', axis=1)
properti_price2.head(100)

In [ ]:
properti_price2.info

In [ ]:
properti_price2.describe()

In [ ]:
print(properti_price2['price'].value_counts())

In [ ]:
print("data null \n", properti_price2.isnull().sum())
print("data kosong \n", properti_price2.empty)
print("data nan \n", properti_price2.isna().sum())

In [ ]:
properti_price2.price.plot(kind='box')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
def remove_outlier(df_in):
  for col_name in list(df_in):
    if is_numeric_dtype(df_in[col_name]):
      q1 = df_in[col_name].quantile(0.25)
      q3 = df_in[col_name].quantile(0.75)

      iqr = q3-q1
      batas_atas = q3+(iqr*1.5)
      batas_bawah = q1-(iqr*1.5)

      df_out = df_in.loc[(df_in[col_name]>=batas_bawah) & (df_in[col_name]<=batas_atas)]

  return df_out

properti_price_clean = remove_outlier(properti_price2)
print("Jumlah baris DataFrame sebelum di outlier",properti_price2.shape[0])
print("Jumlah baris DataFrame sesudah di outlier",properti_price_clean.shape[0])
properti_price_clean.price.plot(kind='box', vert=True)

plt.gca().invert_yaxis()
plt.show()

In [ ]:
print("data null \n", properti_price_clean.isnull().sum())
print("data kosong \n", properti_price_clean.empty)
print("data nan \n", properti_price_clean.isna().sum())

In [11]:
X_regress=properti_price_clean.drop('price',axis=1)
y_regress=properti_price_clean.price

X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(X_regress, y_regress, test_size=0.20, random_state=84)

In [ ]:
X_regress=properti_price_clean.drop('price',axis=1)
y_regress=properti_price_clean.price

X_train_ins, X_test_ins, y_train_ins, y_test_ins = train_test_split(X_regress, y_regress, test_size=0.20, random_state=84)
cat_cols = X_train_ins.select_dtypes(include=['object']).columns.tolist()
print("Kolom kategorik:",cat_cols)

transformer = make_column_transformer(
    (OneHotEncoder(), cat_cols),
    remainder = 'passthrough'
)

X_train_enc = transformer.fit_transform(X_train_ins)
X_test_enc = transformer.transform(X_test_ins)

df_train_enc = pd.DataFrame (X_train_enc, columns=transformer.get_feature_names_out())
df_test_enc = pd.DataFrame (X_test_enc, columns=transformer.get_feature_names_out())

df_train_enc.head(10)
df_test_enc.head(10)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error

pipe_Lasso_KBest = Pipeline(steps=[
            ('scale', StandardScaler()),
            ('feature_selection', SelectKBest(score_func=f_regression)),
            ('reg', Lasso(max_iter=1000)) #max_iter digunakan untuk menen
            ])

param_grid_Lasso_KBest = {
    'reg__alpha': [0.01,0.1,1,10,100],
    'feature_selection__k': np.arange(1,20)
}

GSCV_Lasso = GridSearchCV(pipe_Lasso_KBest, param_grid_Lasso_KBest, cv=5, scoring='neg_mean_squared_error')

GSCV_Lasso.fit(X_train_enc, y_train_price)
print("Best model:{}".format(GSCV_Lasso.best_estimator_))
print("Lasso best parameters: {}".format(GSCV_Lasso.best_params_))

print("Koefisien/bobot:{}".format(GSCV_Lasso.best_estimator_.named_steps['reg'].coef_))
print("Intercept/bias:{}".format(GSCV_Lasso.best_estimator_.named_steps['reg'].intercept_))

Lasso_predict = GSCV_Lasso.predict(X_test_enc)

mse_Lasso = mean_squared_error(y_test_price, Lasso_predict)
mae_Lasso = mean_absolute_error(y_test_price, Lasso_predict)

print("Lasso Mean Squared Error (MSE): {}".format(mse_Lasso))
print("Lasso Mean Absolute Error (MAE): {}".format(mae_Lasso))
print("Lasso Root Mean Squared Error: {}".format(np.sqrt(mse_Lasso)))

In [ ]:
# df_results['Lasso KBest Prediction']=Lasso_predict
df_results = pd.DataFrame(y_test_price)
df_results['Lasso KBest Prediction']=Lasso_predict

df_results['Selisih Price Lasso KBest'] = df_results['Lasso KBest Prediction'] - df_results['price'] 
df_results.head()

In [ ]:
df_results.describe()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error

pipe_Lasso_percentile = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('feature_selection', SelectPercentile(score_func=f_regression)),  # Menggunakan SelectPercentile
    ('reg', Lasso(max_iter=1000))
])

param_grid_Lasso_percentile = {
    'reg__alpha': [0.01, 0.1, 1, 10, 100],
    'feature_selection__percentile': np.arange(10, 100, 10)  # Menggunakan persentase fitur terbaik
}

GSCV_Lasso = GridSearchCV(pipe_Lasso_percentile, param_grid_Lasso_percentile, cv=5, scoring='neg_mean_squared_error')

# Fit ke data latih
GSCV_Lasso.fit(X_train_enc, y_train_price)

# Hasil dari GridSearch
print("Best model:{}".format(GSCV_Lasso.best_estimator_))
print("Lasso best parameters: {}".format(GSCV_Lasso.best_params_))

# Koefisien dan intercept dari model terbaik
print("Koefisien/bobot:{}".format(GSCV_Lasso.best_estimator_.named_steps['reg'].coef_))
print("Intercept/bias:{}".format(GSCV_Lasso.best_estimator_.named_steps['reg'].intercept_))

# Prediksi terhadap data uji
Lasso_predict = GSCV_Lasso.predict(X_test_enc)

# Menghitung error
mse_Lasso = mean_squared_error(y_test_price, Lasso_predict)
mae_Lasso = mean_absolute_error(y_test_price, Lasso_predict)

print("Lasso Mean Squared Error (MSE): {}".format(mse_Lasso))
print("Lasso Mean Absolute Error (MAE): {}".format(mae_Lasso))
print("Lasso Root Mean Squared Error: {}".format(np.sqrt(mse_Lasso)))


In [ ]:
df_results['Lasso Percentile Prediction']=Lasso_predict
df_results = pd.DataFrame(y_test_price)
df_results['Lasso Percentile Prediction']=Lasso_predict

df_results['Selisih Price Lasso Percentile'] = df_results['Lasso Percentile Prediction'] - df_results['price'] 
df_results.head()

In [ ]:
df_results.describe()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Pipeline for Random Forest Regressor
pipe_RF = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('feature_selection', SelectPercentile(score_func=f_regression)),  # Pilih top percentile fitur
    ('reg', RandomForestRegressor(random_state=84))  # Random Forest Regressor
])

# Parameter grid untuk GridSearchCV
param_grid_RF = {
    'reg__n_estimators': [100, 200],          # Jumlah pohon lebih sedikit
    'reg__max_depth': [2,3],         # Variasi kedalaman terbatas
    'feature_selection__percentile': np.arange(10, 50)  # Langkah 10 untuk persentil
}

# GridSearchCV to find the best parameters
GSCV_RF = GridSearchCV(pipe_RF, param_grid_RF, cv=5, scoring='neg_mean_squared_error')

# Fit to the training data
GSCV_RF.fit(X_train_enc, y_train_price)

# Best model and parameters
print("Best model:{}".format(GSCV_RF.best_estimator_))
print("Random Forest best parameters: {}".format(GSCV_RF.best_params_))

# Make predictions on the test set
RF_predict = GSCV_RF.predict(X_test_enc)

# Calculate metrics
mse_RF = mean_squared_error(y_test_price, RF_predict)
mae_RF = mean_absolute_error(y_test_price, RF_predict)

print("Random Forest Mean Squared Error (MSE): {}".format(mse_RF))
print("Random Forest Mean Absolute Error (MAE): {}".format(mae_RF))
print("Random Forest Root Mean Squared Error: {}".format(np.sqrt(mse_RF)))


In [ ]:
df_results['Random Forest Percentile Prediction']=RF_predict
df_results = pd.DataFrame(y_test_price)
df_results['Random Forest Percentile Prediction']=RF_predict

df_results['Selisih Price RF Percentile'] = df_results['Random Forest Percentile Prediction'] - df_results['price'] 
df_results.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Pipeline for Random Forest Regressor
pipe_RF_Kbest = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('feature_selection', SelectKBest(score_func=f_regression)),  # Pilih top percentile fitur
    ('reg', RandomForestRegressor(random_state=84))  # Random Forest Regressor
])

# Parameter grid untuk GridSearchCV
param_grid_RF_Kbest = {
    'reg__n_estimators': [100, 200],          # Jumlah pohon lebih sedikit
    'reg__max_depth': [2,3],         # Variasi kedalaman terbatas
    'feature_selection__k': np.arange(10, 50)  # Langkah 10 untuk persentil
}

# GridSearchCV to find the best parameters
GSCV_RF = GridSearchCV(pipe_RF_Kbest, param_grid_RF_Kbest, cv=5, scoring='neg_mean_squared_error')

# Fit to the training data
GSCV_RF.fit(X_train_enc, y_train_price)

# Best model and parameters
print("Best model:{}".format(GSCV_RF.best_estimator_))
print("Random Forest best parameters: {}".format(GSCV_RF.best_params_))

# Make predictions on the test set
RF_predict = GSCV_RF.predict(X_test_enc)

# Calculate metrics
mse_RF = mean_squared_error(y_test_price, RF_predict)
mae_RF = mean_absolute_error(y_test_price, RF_predict)

print("Random Forest Mean Squared Error (MSE): {}".format(mse_RF))
print("Random Forest Mean Absolute Error (MAE): {}".format(mae_RF))
print("Random Forest Root Mean Squared Error: {}".format(np.sqrt(mse_RF)))


In [ ]:
df_results['Random Forest KBest Prediction']=RF_predict
df_results = pd.DataFrame(y_test_price)
df_results['Random Forest KBest Prediction']=RF_predict

df_results['Selisih Price RF KBest'] = df_results['Random Forest KBest Prediction'] - df_results['price'] 
df_results.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Misalkan Ridge_predict dan SVR_predict sudah didefinisikan sebelumnya
# Ridge_predict = model_ridge.predict(X_test)
# SVR_predict = model_svr.predict(X_test)

# Mengonversi y_test_price menjadi DataFrame
df_results = pd.DataFrame(y_test_price)

# Menambahkan kolom prediksi
df_results['Lasso KBest Prediction']=Lasso_predict
df_results['Random Forest KBest Prediction']=RF_predict

# Jika ada kolom lain yang perlu ditambahkan
df_results['Lasso Percentile Prediction']=Lasso_predict
df_results['Random Forest Percentile Prediction']=RF_predict

# Menghitung selisih

df_results['Selisih Price Lasso KBest'] = df_results['Lasso KBest Prediction'] - df_results['price']
df_results['Selisih Price RF KBest'] = df_results['Random Forest KBest Prediction'] - df_results['price']

# Menampilkan beberapa data teratas
print(df_results.head())

# Membuat plot
plt.figure(figsize=(20, 5))
data_len = range(len(y_test_price))
plt.scatter(data_len, df_results['price'], label="Actual", color="blue")
plt.plot(data_len, df_results['Lasso KBest Prediction'], label="Lasso KBest Prediction", color="green", linewidth=1, linestyle="dashed")
plt.plot(data_len, df_results['Lasso Percentile Prediction'], label="Lasso Percentile Prediction", color="red", linewidth=1, linestyle="dashed")
plt.plot(data_len, df_results['Random Forest KBest Prediction'], label="Random Forest KBest Prediction", color="yellow", linewidth=1, linestyle="-.")
plt.plot(data_len, df_results['Random Forest Percentile Prediction'], label="Random Forest Percentile Prediction", color="black", linewidth=1, linestyle="-.")

# Menambahkan legenda dan menampilkan plot
plt.legend()
plt.show()

In [ ]:
import pickle
best_model = GSCV_RF.best_estimator_

with open('BestModel_REG_GSCV_RF_matplotlib.pkl', 'wb') as f:
    pickle.dump(best_model, f)
print("Model Terbaik berhasil disimpan ke 'BestModel_REG_GSCV_RF_matplotlib.pkl")